# Chapter 1
Covers the concepts of __primitives__, __combinations__, and __abstractions__.

- Primitive: a symbol directly manipulable by the language, ex. integers, addition
- Combination: a means of combining epxressions of primitives to make complex expressions
- Abstraction: a way to label combinations and use them as primitives in a more complex problem

The meaning of symbols in these combinatorial expressions, it is said, are determined by the __global environment__.

All expressions by default are evaluated as such: the first item in the expression is the operand, and operates on all the items in the expression. Special forms are introduced which are exceptions to this rule. In order for the expression to be evaluated, its operands are evaluated, then the result is substituted into the definition of the operator (recursively repeating this process until only primitives remain), and then the final primitive expression is evaluated. This is called __applicative order__ because the operator is applied to the results of evaluating the operands.

The other means of evaluating expressions is to first substitute the operand values into the operand, and then evaluate the result. That is, take the values for parameters (unevaluated), and substitute them in for the operator's parameters until only primitives remain, and evaluate the resulting expression. This is called __normal order__ evaluation. In __normal order__ evaluation, we will repeat evaluations for operand expressions if the corresponding parameter appears multiple times in the operator's definition.

## Ordinary procedures
- `not` because the value of the operand is always evaluated

## Special Forms
- `define` is used to create new operands
- `cond` is used to conditionally evaluate expressions. Only the first expression whose predicate is true is evaluated, and predicates are evaluated in order
- `if` is used to conditionally evaluate one of two expressions, depending on the result of evaluating the predicate (if true, evaluate the consequent, otherwise the alternative)
- `and` evaluate the operands until one evaluates to false or none are left, and the value of the expression is the result of the most recently evaluated operand
- `or` evaluate the operands until one evaluates to a true value or none are left, and return the value of the most recently evaluated expression

__Note:__ in order to avoid a bug in calysto scheme's lexical scope code, the following is required a work-around, as described [here](https://github.com/Calysto/calysto_scheme/issues/8).

In [1]:
(use-lexical-address #f)

## Exercise 1.1
The number 10 simply evaluates to `10`

In [2]:
10

10

This expression sums the three numbers since operators may take an arbitrary number of operands. The result is `12`

In [3]:
(+ 5 4 3)

12

This expression is a simple subtraction; the result is `8`

In [4]:
(- 9 1)

8

This is an integer division problem whose result is `3`

In [5]:
(/ 6 2)

3

This is a compound expression. The operator is a primitive, so no substitution is required. The operands evaluate to `8` and `-2` respectively, so the result is `6`

In [6]:
(+ (* 2 4) (- 4 6))

6

This is the first example of a special form. Define does not evaluate either `a` or the primitive `3`, but instead creates a new symbol in the environment called `a`, whose definition is the primitive `3`. This result is not accompanied by any output, but does affect our use of the symbol `a` in future expressions.

In [7]:
(define a 3)

This expression defines a new symbol which is equal to the evaluation of the second operand, `(+ a 1)`, which is the same as the arithmetical expression `a + 1`, which is `4`.

In [8]:
(define b (+ a 1))

This expression retrieves the values of `a` and `b` from the environment, and uses them to evaluate first the operand `(* a b)` to `(* 3 4)`, which is `12`, and then the complete expression `(+ a b 12)`, which is `(+ 3 4 12)`, evaluates to `19`.

In [9]:
(+ a b (* a b))

19

This expression is a predicate, since it evaluates to "false", which in Scheme is represented by the symbol `#f`

In [10]:
(= a b)

#f

Now we have our second special form: `if`. This expression first evaluates the predicate to see that `b` is greater than `a`, and `b` is less than `b * a`, and since the predicate is true, the consequent is then evaluated to `b`, which is `4`, so the value of the whole expression is `4`.

In [11]:
(if (and (> b a) (< b (* a b)))
    b
    a)

4

This expression uses the special form `cond`. The first predicate is evaluated, and since `a` is not equal to `4`, evaluation continues. The second predicate is evaluated, and found to be true, so the value of the whole expression is equal to the evaluation of the second consequent, which is equal to `16`

In [12]:
(cond ((= a 4) 6)
      ((= b 4) (+ 6 7 a))
      (else 25))

16

This expression uses the special form `if` as an operand. The `if` statement evaluates to `b`, and then `b` is added to `2` to get `6`

In [13]:
(+ 2 (if (> b a) b a))

6

Similar to the above, the `cond` expression evaluates to `b`, which is the only consequent in the `cond` expression to be evaluated. This result is multiplied by `a + 1` to get `16`

In [14]:
(* (cond ((> a b) a)
         ((< a b) b)
         (else -1))
   (+ a 1))

16

## Exercise 1.2
This problem asks us to go from standard arithmetic notation to prefix notation. In order to do so, we must use the order of operations. The fraction bar is the least tightly bound operator, so it goes first. The groupings in the numerator are all unnecessary, since addition is associative. On the bottom, the multiplication binds less tightly than the groupings, so the two subtractions must take place first, i.e. the multiplication comes first in prefix notation.

I am reading the expression to be: $$ \frac{5 + 4 + (2 - (3 - (6 + \frac{4}{5}))}{3 (6 - 2) (2 - 7)} $$

To check our work, we note that the arithmetic expression evaluates to: 
$$ \frac{5 + 4 + 2 - (3 - (6 + 4/5))}{3(6 - 2)(2 - 7)} $$ 

$$ = \frac{-37}{150} $$ 

$$ = -0.247 $$

In [15]:
(/ (- (+ 5 4 2)
      (- 3 (+ 6 (/ 4 5))))
   (* 3
      (- 6 2)
      (- 2 7)))

-37/150

## Exercise 1.3
The procedure below is to return the sum of the squares of the larger two of a group of three numbers. I use the procedure `sum-of-squares` from the text.

In [16]:
(define (square x) (* x x))
(define (sum-of-squares x y)
  (+ (square x) (square y)))

(define (f a b c) 
  (cond ((and (<= a b) (<= a c)) (sum-of-squares b c))
        ((and (<= b a) (<= b c)) (sum-of-squares a c))
        ((and (<= c a) (<= c b)) (sum-of-squares a b))))

As an example, the sum-of-squares of `3` and `4` is `25`, and so `(f 2 3 4)` should return `25`

In [17]:
(f 2 3 4)

25

## Exercise 1.4
I use the definition of `abs` from the chapter to test the behavior of the following routine according to the semanics. The expectation is that the `if` statement chooses the operator based on the value of `b`, and then uses the resulting operator to evaluate the whole expression. Thus we expect the predicate to evaluate to "true", which in scheme is represented as `#t`

In [18]:
(define (abs x)
  (if (< x 0)
      (- x)
      x))

(define (a-plus-abs-b a b)
  ((if (> b 0) + -) a b))

(= (+ 1 (abs -2))
   (a-plus-abs-b 1 -2))

#t

## Exercise 1.5
In either evaluation order, we are to treat the `if` statement as evaluating the same. Thus in either evaluation order, the `if` statement will find the condition to be true, and evaluate to the consequent, or `0`. 

The difference is, in an __applicative order__ interpreter, the operands of `test` will be evaluated, and `p` will be treated as a procedure with no arguments, which evaluates to itself. This will result in infinite recursion. In a __normal order__ interpreter, the two expressions (`0` and `(p)`), will be substituted into the original definition, and since the `if` statement does not evaluate the alternative, the statement `p` will never be evaluated.

In [19]:
(define (p) (p))

(define (test x y)
  (if (= x 0)
      0
      y))

The test procedure is shown here to work with two integers, showing that it is in fact the definition of `(p)` which causes the infinite loop to occure when `test` is evaluated. That is, the problem would occur with any procedure evaluated in __applicative order__.

In [20]:
(test 0 1)

0

In the following code, `(p)` is quoted (which refers to the list containing the symbol `p` rather than the procedure `(p)` with no arguments), since the scheme interpreter uses __applicative order__, and the result would be an infinite loop which would cause errors with the Jupyter notebook.

In [21]:
(test 0 '(p))

0

## Procedures vs. Functions
The difference here is explained: that a function is declarative, while a procedure is effective. In order to be a procedure, an expression must follow some process to find the answer, rather than simply describe the answer to a question. As an example, the following are given as a definition of the square root.

In [22]:
(define (sqrt-iter guess x)
  (if (good-enough? guess x)
      guess
      (sqrt-iter (improve guess x)
                 x)))

(define (improve guess x)
  (average guess (/ x guess)))

(define (average x y)
  (/ (+ x y) 2))

(define (good-enough? guess x)
  (< (abs (- (square guess) x)) 0.001))

(define (sqrt x)
  (sqrt-iter 1.0 x))

(sqrt 9)

3.00009155413138

## Exercise 1.6
Alyssa's `new-if` procedure seems at first glance to work just like `if`. In fact, if the logic inside of `new-if` were used in place of `if`, then it would work exactly the same. The problem with the `new-if` construct is parallel to the issue in Exercise 1.5 - the __applicative order__ evaluation causes both the consequent and the alternative to be evaluated before being passed to the `new-if` procedure. This causes our `square-root` procedure to infinitely evaluate the expression `(sqrt-iter (improve guess x) x))`.

To demonstrate this, I'll show that using the `cond` logic works. However, demonstrating the infinite regress of the `new-if` statement would break the notebook, so it will be "left as an exercise to the reader".

In [23]:
(define (new-if predicate then-clause else-clause)
  (cond (predicate then-clause)
        (else else-clause)))

(new-if (= 2 3) 0 5)

5

In [24]:
(new-if (= 1 1) 0 5)

0

It can be seen that the `cond` statement below is equivalent to the definition of `new-if`, and the problem arises from the application of `new-if`, rather than the logic or evaluation order of `cond`.

Using __normal order__ evaluation, we would find that applying `new-if` to the operands of:

would result in the following bindings:

which is where the following expression comes from.

In [25]:
(define (sqrt-iter guess x)
  (cond ((good-enough? guess x) guess)
        (else (sqrt-iter (improve guess x) x))))

(sqrt 9)

3.00009155413138

## Exercise 1.8
In order to adapt our `sqrt` procedure to compute cubed roots, the only aspect which needs to change is the `improve` procedure. Since we will implement Newton's method more generally as a later problem according to the text, I'll just re-write `improve` and `sqrt-iter` and `sqrt` here for cube roots.

This is really just an exercise in prefix notation, as well as a chance to work out the logic of the `sqrt` procedure for oneself.

In [26]:
(define (cube x) (* x x x))

(define (cbrt-iter guess x)
  (if (good-enough-cbrt? guess x)
      guess
      (cbrt-iter (improve-cbrt guess x)
                 x)))

(define (good-enough-cbrt? guess x)
  (< (abs (- (cube guess) x)) 0.001))

(define (improve-cbrt guess x)
  (/ (+ (/ x
           (square guess))
        (* 2 guess)) 
     3))

(define (cbrt x)
  (cbrt-iter 1.0 x))

(cbrt 27)

3.0000005410641766

## Recursion
The remainder of the chapter explains the basics of recursion. One noteworthy abstraction is the procedure graph, which shows the procedures composing `sqrt`, the procedures composing those, and so on. This shows that each procedure is build of smaller components, which we do not need to completely understand to be able to comprehend the process of evaluating the `sqrt` procedure.

__Bound__ and __free__ variables are also discussed. __Bound__ variables are those which are declared in the first operand of the `define` special form. These have a certain value in the scope of the procedure which is determined at the time the procedure is called. __Free__ variables, on the other hand, are not set to a determined value, but instead are defined by the larger environment, or scope. For example, in `good-enough?`, `abs` is defined above. It is also noteworthy that we can name a __bound__ variable in such a way as to co-opt a named variable in a larger scope without affecting the accessors of that variable outside our current scope.

Building on this idea, we can use what is called __block structure__, where we put definitions of helper procedures inside the parent procedure. In this case, the eample given is:

In [27]:
(define (sqrt x)
  (define (sqare x) 
    (* x x))
  (define (average x y)
    (/ (+ x y) 2))
  (define (good-enough? guess x)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess x)
    (average guess (/ x guess)))
  (define (sqrt-iter guess x)
    (if (good-enough? guess x)
        guess
        (sqrt-iter (improve guess x) x)))
  (sqrt-iter 1.0 x))

(sqrt 9)

3.00009155413138

Next, it is observed that the variable `x` has the same meaning throughout the procedure `sqrt`'s definition. Therefore the concept of __lexical scoping__ is introduced, where if a __bound__ variable is going to have the same value as its value in the parent scope, we can simply remove the __bound__ variable from the parameters list and let this value be used in the child definitions, making it a __free__ variable in the child procedure while remaining __bound__ by the parent.

In [28]:
(define (sqrt x)
  (define (square y)
    (* y y))
  (define (average a b)
    (/ (+ a b) 2))
  (define (good-enough? guess)
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))

(sqrt 9)

3.00009155413138

I would like to improve on this definition, which doesn't quite match the book so that it may definitely be run without necessarily running the above definitions of `average` and `square`, by fully binding the variables `guess` and `x` in the definitions of `average` and `square`.

In [29]:
(define (sqrt x)
  (define (good-enough? guess)
    (define (square y)
      (* y y))
    (< (abs (- (square guess) x)) 0.001))
  (define (improve guess)
    (define (average a b)
      (/ (+ a b) 2))
    (average guess (/ x guess)))
  (define (sqrt-iter guess)
    (if (good-enough? guess)
        guess
        (sqrt-iter (improve guess))))
  (sqrt-iter 1.0))

(sqrt 9)

3.00009155413138

## Linear Recursion and Iteration
The difference between __linear recursion__ and __iteration__ is that the returned value from a __linear recursive__ procedure is different on each step, so the full call pattern (which is the state) is pushed onto the call stack, which can cause buffer overflows. With __iteration__, the state of the procedure is contained in the operands, so the values do not need to be pushed on to the stack with successive calls.

A __linear recursive__ procedure definition of `factorial` is below.

In [30]:
(define (factorial n)
  (if (= n 1)
      1
      (* n (factorial (- n 1)))))

(factorial 6)

720

The below definition, on the other hand, is an __iterative__ procedure.

In [31]:
(define (factorial n)
  (fact-iter 1 1 n))

(define (fact-iter acc count n)
  (if (> count n)
      acc
      (fact-iter (* acc count)
                 (+ count 1)
                 n)))

(factorial 6)

720

This new definition of factorial will not buffer overflow because of Scheme's use of __tail recursion__. This simply means that the implementation of Scheme allows for __iterative__ _processes_ to be implemented as __recursive__ _procedures_, without adding frames to the call stack on each iteration.

## Exercise 1.9
The two procedures are shown below, with helper methods. The procedures here are re-named `new+` and `new+tail` to avoid errors due to re-defining the `+` procedure.

In [32]:
(define (inc x)
  (+ x 1))
(define (dec x)
  (- x 1))

(define (new+ a b)
  (if (= a 0)
      b
      (inc (new+ (dec a) b))))

(define (new+tail a b)
  (if (= a 0)
      b
      (new+tail (dec a) (inc b))))

The processes generated by these procedures are shown here.

This process is __linear recursive__, since the state of the addition is hidden in the interpreter. Each time the procedure is called, its result is evaluated as an operand of the calling procedure.

This process is __iterative__, because the procedure is not called in the operands, but the procedure returns the same value returned by itself, with different operands. This is what is meant in the text by "The state is contained in the operands of the procedure."

## Exercise 1.10
The procedure for Ackerman's function from the book is below.

In [33]:
(define (A x y)
  (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))

The first expression from the exercise evaluates to 1024. The process illustration, as in the book is:

In [34]:
(A 1 10)

1024

The second expression evaluates to 65536. The process illustartion is as follows, with steps that have already been computed in the above section omitted.

In [35]:
(A 2 4)

65536

Doing some research on the Ackermann function, it seems that it computes a meta-multiplication by `2`. That is, 

`(A 0 n)` $ = 2*n $, which is `f`

`(A 1 n)` $ = 2*2$ n times $ = 2^n $, which if `g`

`(A 2 n)` $ = 2^{{2^2}^{...}} $ n times, which we'll call: $ h(n) $, and which is `h` from the example

`(A 3 n)` $ = h(h(...h(1))) $ n times

Going off of this, we expect `(A 3 3)` to be $ h(h(h(1))) $, which would give us

$$ = h(h(2)) $$

$$ = h(2^2) $$

$$ = h(4) $$

$$ = 2^{2^{2^{2}}} $$

$$ = 65536 $$

... clearly this method of computation will get out of hand very quickly with increasing values of `x`.

In [36]:
(A 3 3)

65536

The next section covers __tree-recursion__, which is a recursive process whose procedure is a part of more than one operand in its own definition. The example given is `fib`.

In [37]:
(define (fib n)
  (cond ((= n 0) 0)
        ((= n 1) 1)
        (else (+ (fib (- n 1))
                 (fib (- n 2))))))

(fib 5)

5

Next is an example of a linear definition of `fib`, which takes advantage of the fact that we only need to track two numbers at a time to determine the next fibonacci number.

In [38]:
(define (fib n)
  (fib-iter 1 0 n))

(define (fib-iter a b count)
  (if (= count 0)
      b
      (fib-iter (+ a b) a (- count 1))))

(fib 5)

5

An example of a __tree-recursive__ process is given: how to count the number of ways to give change with a set of coins. The point made is that this __tree-recursive__ process is easy to demarcate, but it is difficult to find an __iterative__ process to do the same thing.

In [39]:
(define (count-change amount)
  (cc amount 5))
(define (cc amount kinds-of-coins)
  (cond ((= amount 0) 1)
        ((or (< amount 0) (= kinds-of-coins 0)) 0)
        (else (+ (cc amount
                     (- kinds-of-coins 1))
                 (cc (- amount
                        (first-denomination kinds-of-coins))
                     kinds-of-coins)))))
(define (first-denomination kinds-of-coins)
  (cond ((= kinds-of-coins 1) 1)
        ((= kinds-of-coins 2) 5)
        ((= kinds-of-coins 3) 10)
        ((= kinds-of-coins 4) 25)
        ((= kinds-of-coins 5) 50)))

(count-change 100)

292

## Exercise 1.11
A recursive process for this function is easy to write a procedure for, since it basically means re-writing the formula in LISP.

$ f(n) = n $ if $ n<3 $ and $ f(n) = f(n - 1) + 2f(n - 2) + 3f(n - 3) $ if $ n\ge3 $

In [40]:
(define (f n)
  (if (< n 3)
      n
      (+ (f (- n 1))
         (* 2 (f (- n 2)))
         (* 3 (f (- n 3))))))

(f 3)

4

An iterative process is somewhat more difficult. In order to capture the process, we can store the results of previous computations in accumulative operands.

In [41]:
(define (f-iter n)
  (if (< n 3)
      n
      (fi 2 1 0 3 n)))

(define (fi fn-minus-1 fn-minus-2 fn-minus-3 acc n)
  (define fn
    (+ fn-minus-1
         (* 2 fn-minus-2)
         (* 3 fn-minus-3)))
  (if (= acc n)
      fn
      (fi fn
          fn-minus-1
          fn-minus-2
          (+ acc 1)
          n)))

(f-iter 3)

4

In [42]:
;; Test to see that f and f-iter give the same results
(and (= (f-iter -10)
        (f -10))
     (= (f-iter 4)
        (f 4))
     (= (f-iter 5)
        (f 5))
     (= (f-iter 6)
        (f 6))
     (= (f-iter 15)
        (f 15)))

#t

Now we look for a procedure to define Pasca's Triangle. Since we need to determine most of the triangle above an element in order to determine that element, a recursive procedure makes sense. We'll use a __tree-recursive__ process since it reflects the definition of Pascal's Triangle and so will be much easier to write.

In [43]:
(define (pascal row col)
  (cond ((or (< col 1) (< row 1) (> col row)) 0)
        ((or (= row 1) (= row 2)) 1)
        (else (+ (pascal (- row 1) (- col 1))
                 (pascal (- row 1) col)))))

(pascal 5 3)

6

## Exercise 1.13
To prove that $ Fib(n) = round(\frac{\phi^n}{\sqrt5}) $ using deduction, we first need to show, using the hint from the exercise, that $ Fib(1) = round(\frac{\phi}{\sqrt5}) $ and $ Fib(2) = round(\frac{\phi^2}{\sqrt5}) $.

In [44]:
(define phi (/ (+ (sqrt 5) 1) 2))
(define psi (/ (- 1 (sqrt 5)) 2))

phi

1.6180344478216817

In [45]:
(define (pow x n)
  (define (pow-iter x n acc)
    (if (= n 0)
        acc
        (pow-iter x (- n 1) (* acc x))))
  (pow-iter x n 1))

(pow 2 5) ;; Expect 2^5 = 32

32

In [46]:
(define (fib-approx n)
  (/ (pow phi n) (sqrt 5)))

(fib-approx 1)

0.7236067059356593

In [47]:
(fib 1)

1

In [48]:
(fib-approx 2)

1.1708205768786706

In [49]:
(fib 2)

1

So now we have shown that $ Fib(1) = round(\frac{\phi}{\sqrt5}) $ and $ Fib(2) = round(\frac{\phi^2}{\sqrt5}) $.

Now we must show that $ round(\frac{\phi^{n + 2}}{\sqrt5}) = round(\frac{\phi^{n + 1}}{\sqrt5}) + round(\frac{\phi^{n}}{\sqrt5}) $. This is very difficult to do since we do not know what direction the $round$ operation will move the result, either up or down. So instead, we use the hint from the text again and show that the $\phi$ and $\psi$ definition of $Fib$ gives us the exact number.

In [50]:
(define (fib-phi-psi n)
  (/ (- (pow phi n)
        (pow psi n))
     (sqrt 5)))

(fib-phi-psi 1)

1.0

For the $ n = 1 $ case:

$$ \frac{1}{\sqrt5} \bigg( \frac{1 + \sqrt5}{2} - \frac{1 - \sqrt5}{2} \bigg) $$

$$ = \frac{1}{\sqrt5} * \frac{2 * \sqrt5}{2} $$

$$ = 1 $$

For the $ n = 2 $ case:

$$ \frac{1}{\sqrt5} \bigg( \frac{(1 + \sqrt5)(1 + \sqrt5)}{2} - \frac{(1 - \sqrt5)(1 - \sqrt5)}{2} \bigg) $$

$$ = \frac{1}{\sqrt5} \bigg( \frac{1 + 2\sqrt5 + 5}{2} - \frac{1 - 2\sqrt5 + 5}{2} \bigg)$$

$$ = \frac{1}{\sqrt5} \bigg( \frac{4\sqrt5}{2} \bigg) $$

For the $ n > 2 $ case:

$$ \frac{1}{\sqrt5} \bigg( \frac{(1 + \sqrt5)^{n + 2}}{2^{n+2}} - \frac{(1 - \sqrt5)^{n + 2}}{2^{n+2}} \bigg) $$
- expand out the square terms in the numerators

$$ = \frac{1}{\sqrt5} \bigg( \frac{(1 + \sqrt5)^n(1+2\sqrt5+5)}{2^{n+2}} - \frac{(1 - \sqrt5)^n(1+2\sqrt5+5)}{2^{n+2}} \bigg) $$

$$ = \frac{1}{\sqrt5} \bigg( \frac{(1 + \sqrt5)^n(6+2\sqrt5)}{2^{n+2}} - \frac{(1 - \sqrt5)^n(6+2\sqrt5)}{2^{n+2}} \bigg) $$
- separate the 6 term in both numerators to $(4 + 2)$

$$ = \frac{1}{\sqrt5} \bigg( \frac{4(1 + \sqrt5)^n - 4(1 - \sqrt5)^n}{2^{n+2}} + \frac{(1 + \sqrt5)^n(2+2\sqrt5) - (1 - \sqrt5)^n(2+2\sqrt5)}{2^{n+2}} \bigg) $$
- now partially factor the denominators

$$ = \frac{1}{\sqrt5} \bigg( \frac{4((1 + \sqrt5)^n - (1 - \sqrt5)^n)}{4*2^n} + \frac{2(1 + \sqrt5)^n(1+1\sqrt5) - 2(1 - \sqrt5)^n(1+1\sqrt5)}{2*2^{n+1}} \bigg) $$

$$ = \frac{1}{\sqrt5} \bigg( \frac{(1 + \sqrt5)^n - (1 - \sqrt5)^n}{2^n} \frac{(1 + \sqrt5)^{n+1} - (1 - \sqrt5)^{n+1}}{2^{n+1}} \bigg) $$

$$ = \frac{1}{\sqrt5} \bigg( (\phi^n - \psi^n) + (\phi^{n+1} - \psi^{n+1}) \bigg) $$

$$ = \frac{(\phi^n - \psi^n)}{\sqrt5} + \frac{(\phi^{n+1} - \psi^{n+1})}{\sqrt5} $$

This is the definition of the Fibonacci sequence, and so by deduction, this definition holds for $ n \ge 1$.

## Exercise 1.14

The process diagram of the count-change process is:

The tree illustration is drawn below, but I'll make no claims on my penmanship.

![ch1-tree.jpg](images/ch1-tree.jpg)

In [51]:
(count-change 11)

4

## Exercise 1.15
With the recursive definition of `sine` as shown below, 

In [52]:
(define (sine angle)
  (define (cube x) (* x x x))
  (define (p x) (- (* 3 x) (* 4 (cube x))))
  (if (not (> (abs angle) 0.1))
      angle
      (p (sine (/ angle 3.0)))))

(sine 12.15)

-0.39980345741334

... how many times is the procedure `p` called?

This question can be rephrased as "How many times must `12.15` be divided by `3` in order to get a number `< 0.1`?" In arithmetic notation, this is the same as "What is the smallest integer value of $n$?"

$$ \frac{12.15}{3^n} < 0.1 $$

$$ \implies 12.15 < 0.1 * 3^n $$

$$ \implies 121.5 < 3^n $$

$$ \implies log(121.5) < log(3^n) $$

$$ \implies log(121.5) < n * log(3) $$

$$ \implies \frac{log(121.5)}{log(3)} < n $$

$$ \implies log_3(121.5) < n $$

$$ \implies n > 4.36... $$

The smallest integer $n$ which holds is $ n = 5 $

The next part of the question asks how quickly the space and number of steps increaces with `a` for the process generated by `(sine a)`.

Since the result of a call to `sine` is either its argument, or the result of a call to `p`, and not a call to `sine`, the process is not rail recursive. This means that the space will grow linearly with the number of calls to `sine`, like so...

```
(sine <large-number>)
(p (sine (/ <large-number> 3)))
(p (sine <large-number-by-3>))
(p (p (sine (/ <large-number-by-3> 3))))
(p (p (sine <large-number-by-9>)))
...
(p (p ... (p (p <some-number>)...)))
(p (p ... (p (- (* 3 <some-number>) (* 4 (cube <some-number>))))))
...
(p (p ... (p <some-other-number>)))
```

So as `a` increaces, we expect the number of calls to `p` by the procedure to increace by `(/ a 3)`. Likewise, the required space will increase by the same factor. 

## Excercise 1.16
The hint from the exercise makes our work somewhat easier. The hint says to use the invariant $ a b^n $ to guide our procedure, meaning that we change $a$ and $n$ at each call in order to keep the expression constant, so that at the end of the call pattern, all values are known and one of the values is the answer we are looking for.

In [53]:
(define (fast-expt b n)
  (fast-expt-iter 1 b n))

;; Invariant -> a * b^n
(define (fast-expt-iter a b n)
  (define (square m) (* m m))
  (cond ((= n 0) a)
        ((even? n) (fast-expt-iter a
                                   (square b)
                                   (/ n 2)))
        (else (fast-expt-iter (* a b)
                              b
                              (- n 1)))))

(= 32 (fast-expt 2 5)) ;; Test that 2^5==32

#t

## Exercise 1.17
Replace the existing `*` primitive with a procedure which, like `fast-expt`, uses a logarithmic amount of calls (though not a logarithmic amount of space) when generating a process.

In [54]:
(define (new* a b)
  (if (= b 0)
      0
      (+ a (new* a (- b 1)))))

(= (new* 5 3) 15)

#t

In [55]:
(define (fast-new* a n)
  (define (double m) (+ m m))
  (define (halve m) (/ m 2))
  (cond ((= n 0) 0)                                  ;; a * 0 == 0
        ((= n 1) a)                                  ;; a * 1 == a
        ((even? n) (double (fast-new* a (halve n)))) ;; a * 2n == 2(a * n)
        (else (+ a (fast-new* a (- n 1))))))         ;; a * (2n + 1) == a + a * 2n

(and (= (fast-new* 5  3) 15)    ;; Test our new procedure, 5*3==15
     (= (fast-new* 0  2) 0)     ;; 0*2==0
     (= (fast-new* 10 0) 0)     ;; 10*0==0
     (= (fast-new* 7  7) 49)    ;; 7*7==49
     (= (fast-new* 1  1) 1))    ;; 1*1==1

#t

## Exercise 1.18
The same as 1.17, but with a logarithmic amount of space.

In [56]:
(define (fast-new* a b)
  (fast-new*-iter 0 a b))

;; Invariant -> n + a b
(define (fast-new*-iter n a b)
  (define (double m) (+ m m))
  (cond ((= b 0) n)
        ((even? b) (fast-new*-iter n
                                   (double a)
                                   (/ b 2)))
        (else (fast-new*-iter (+ n a)
                              a
                              (- b 1)))))

(and (= (fast-new* 5 3) 15)    ;; Test our new procedure, 5*3==15
     (= (fast-new* 0 2) 0)     ;; 0  * 2 == 0
     (= (fast-new* 10 0) 0)    ;; 10 * 0 == 0
     (= (fast-new* 7 7) 49)    ;; 7  * 7 == 49
     (= (fast-new* 1 1) 1))    ;; 1  * 1 == 1

#t

## Exercise 1.19
The special case of $\{a \leftarrow bq + aq + ap, b \leftarrow bp + aq\}$ is $\{q = 1, p = 0\}$.

Applying this transformation twice, we get:

$$ S_0 = (a, b) $$

$$ S_1 = (bq + aq + ap, bp + aq) $$

$$ S_2 = (bq + a(q + p), bp + aq) $$

$$ (q(bp + aq) + (q + p)(bq + aq + ap), p(bp + aq) + q(bq + aq + ap)) $$

$$ = (bpq + aqq + bqq + aqq + apq + bpq + apq + app, bpp + apq + bqq + aqq + apq) $$

$$ = (2bpq + 2aqq + bqq + 2apq + app, bpp + 2apq + bqq + aqq) $$

$$ = (2apq + 2aqq + app + 2bpq + bqq, 2apq + aqq + bpp + bqq) $$

$$ = (a(2apq + 2qq + pp) + b(2pq + qq), a(2pq + qq) + b(pp + qq)) $$

$$ = (a(2pq + qq) + a(pp + qq) + b(2pq + qq), a(2pq + qq) + b(pp + qq)) $$

$$ = (b(2qp + qq) + a(2pq + qq) + a(qq + pp), b(pp + qq) + a(2pq + qq)) $$

Applying the transformation $T_{p'q'}$, which is $a \leftarrow bq' + aq' + ap'$, $b \leftarrow bp' + aq'$, gives the same answer when:

$$ p' = pp + pq $$

$$ q' = 2pq + qq $$

Using this definition of $p'$ and $q'$, we can complete the code given in the text:

In [57]:
(define (log-fib n) ;; Rename to log-fib so we can mantain the defintion of fib
  (fib-iter 1 0 0 1 n))
(define (fib-iter a b p q count)
  (cond ((= count 0) b)
        ((even? count)
         (fib-iter a
                   b
                   (+ (* p p) (* q q))
                   (+ (* q q) (* 2 p q))
                   (/ count 2)))
        (else (fib-iter (+ (* b q) (* a q) (* a p))
                        (+ (* b p) (* a q))
                        p
                        q
                        (- count 1)))))

(and (= (log-fib 1) 1)
     (= (log-fib 2) 1)
     (= (log-fib 3) 2)
     (= (log-fib 4) 3)
     (= (log-fib 5) 5)
     (= (log-fib 6) 8))

#t

## Exercise 1.20
Interpreting the expression `(gcd 206 40)` using normal-order evaluation, and given the definition of `gcd`:

In [58]:
(define (gcd a b)
  (if (= b 0)
      a
      (gcd b (remainder a b))))

we get a process which requires much more space:

In [59]:
(remainder (remainder 206 40) (remainder 40 (remainder 206 40)))

2

As a simple proof that this expansion is correct, the longest line in the expanstion evaluates correctly:

In [60]:
(gcd (remainder (remainder 206 
                           40) 
                (remainder 40 
                           (remainder 206 
                                      40))) 
     (remainder (remainder 40 
                           (remainder 206 
                                      40)) 
                (remainder (remainder 206 
                                      40) 
                           (remainder 40 
                                      (remainder 206 
                                                 40)))))

2

By counting, we see that the `remainder` operation is performed 11 times. For applicative-order evalutation, on the other hand...

By counting, we can see that the `remainder` operation is performed 4 times. The difference $-$ the reason why the `remainder` operation is called fewer times with applicative-order evaluation, is that with normal-order evaluation, the same `remainder` operation is performed multiple times.

```lisp
(remainder (remainder 206 
                           40) 
                (remainder 40 
                           (remainder 206 
                                      40))) 
```
This expression appears in both arguments of the `gcd` expression.